In [ ]:
import pandas as pd
import numpy as np
import dai
import pandas as pd
import matplotlib.pyplot as plt

# 设定参数
instrument = ''  # 填写想要回测的上市公司代码，比如电信: 601728.SH
start_date = '2022-05-01'
end_date = '2025-05-01'
increase_ratio = 0.025 # 上涨比例
decline_ratio = 0.005 # 下跌比例

# 从BigQuant平台获取数据
sql = f"""
SELECT
    date,
    close,
    open,
    volume
FROM
    cn_stock_bar1d
WHERE
    instrument = '{instrument}'
    AND date BETWEEN '{start_date}' AND '{end_date}'
ORDER BY
    date
"""


# 使用DAI执行查询
df = dai.query(sql).df()

# 初始化参数
initial_capital = 1000000
capital_per_position = initial_capital / 5
positions = [0] * 5  # 记录每个仓位的持有情况
cash = initial_capital
shares = 0

# 计算收益
for i in range(1, len(df)):
    previous_close = df['close'].iloc[i - 1]
    current_close = df['close'].iloc[i]
    
    # 买入逻辑
    if current_close <= previous_close * (1 - decline_ratio) and shares < capital_per_position:
        shares_to_buy = capital_per_position / current_close
        shares += shares_to_buy
        cash -= shares_to_buy * current_close
        # print(f"Buy {shares_to_buy} shares at {current_close} on {df['date'].iloc[i]}")
    
    # 卖出逻辑
    elif current_close >= previous_close * (1 + decline_ratio) and shares > 0:
        shares_to_sell = shares / 7
        shares -= shares_to_sell
        cash += shares_to_sell * current_close
        # print(f"Sell {shares_to_sell} shares at {current_close} on {df['date'].iloc[i]}")

# 计算总资产和收益率
final_value = cash + shares * df['close'].iloc[-1]
return_rate = (final_value - initial_capital) / initial_capital * 100
print(f"Final portfolio value: {final_value}, Return rate: {return_rate}%")

# 可视化买卖操作
plt.figure(figsize=(14, 7))
plt.plot(df['date'], df['close'], label='Close Price', color='blue')

# 标记买入点
buy_dates = df['date'][df['close'].diff() < -decline_ratio]
buy_prices = df['close'][df['close'].diff() < -decline_ratio]
plt.scatter(buy_dates, buy_prices, marker='^', color='green', label='Buy Signal', s=100)

# 标记卖出点
sell_dates = df['date'][df['close'].diff() > decline_ratio]
sell_prices = df['close'][df['close'].diff() > decline_ratio]
plt.scatter(sell_dates, sell_prices, marker='v', color='red', label='Sell Signal', s=100)

plt.title(f'Trade Signals for {instrument}')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()

# 计算每日总资产
df['total_value'] = cash + shares * df['close']

# 绘制总收益率曲线
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['total_value'], label='Total Portfolio Value', color='blue')
plt.title('Total Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Total Portfolio Value')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


# 计算年化收益率
annualized_return = (final_value / initial_capital) ** (1 / (len(df) / 252)) - 1

# 计算最大回撤率
rolling_max = df['total_value'].cummax()
drawdown = (df['total_value'] - rolling_max) / rolling_max
max_drawdown = drawdown.min()

# 计算信息比率 (IR)
excess_returns = df['total_value'].pct_change().dropna() - 0.0001  # 假设无风险收益率为0.01%
ir = excess_returns.mean() / excess_returns.std()

# 计算信息系数 (IC)
# 假设当前收益率与前一日收益率的相关性作为IC的计算
ic = df['close'].pct_change().rolling(2).corr(df['close'].pct_change().shift(1)).mean()

metrics = {
    "总收益率": f"{return_rate:.2f}%",
    "年化收益率": f"{annualized_return * 100:.2f}%",
    "最大回撤率": f"{max_drawdown:.2f}%",
    "信息比率 (IR)": f"{ir:.2f}",
    "信息系数 (IC)": f"{ic:.2f}"
}

# 输出Metrics指标
print("\n--- Investment Metrics ---")
for metric, value in metrics.items():
    print(f"{metric}: {value}")